In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
sys.path.append("../BBTRE_physics_analysis/") 

In [3]:
import numpy as np
from xmitgcm import open_mdsdataset 
from xhistogram.xarray import histogram

from osse import *
from canyon_utils import *
from sloped_MITgcm import *

In [5]:
def checklist():
    return f"""Checklist for running variable resolution experiments:

Directory structure:
  [x] Copy input and code files from reference simulation
  [x] Run Jupyter notebook to create input files

/input files:
  data
    [ ] Change iter0 to 0
  data.diagnostics
    [ ] Turn off other Ptracers

[ ] Submit execution batch script
[ ] Check that the job runs succesfully!
"""

In [6]:
Γ = 0.0008613659531090722
N = np.sqrt(g*α*Γ)
θ = 1.26E-3
f = 5.3e-5

h = 230
k0 = 5.2e-5
k1 = 1.8e-3

dx = 600.
delta_t = 15.

In [7]:
data_dir = f"../../MITgcm/experiments/rotated_BBTRE_canyon-tracer/run/"
ds = open_mdsdataset(data_dir,ignore_unknown_vars=True,prefix=["budgetDiag"], delta_t=delta_t)
ds = ds.isel(time=0)
ds, grid = add_rotated_coords(ds, θ)

geo = xr.open_dataset(data_dir+"geocoords.nc")
ds['Depth_actual'] = xr.zeros_like(ds['Depth']) - geo['z_actual'].values

dx = 600.; dy = 600.;
Ly = ds.YC.size*dy
Lx = ds.XC.size*dx

ds.attrs["dx"] = dx;
ds.attrs["dy"] = dy;
ds.attrs["Lx"] = Lx;
ds.attrs["Ly"] = Ly;

ds = ds.assign_coords({"Depth": -ds.Depth})

# Double the background diffusivity
ds["KVDIFF"] = (ds["KVDIFF"] - k0)*2.5 + k0

### Create input files from last snapshot of model output

In [8]:
input_vars = {
    "UVEL": "U",
    "VVEL": "V",
    "THETA": "T",
    "KVDIFF": "kappa",
    "Depth": "topog",
    "drF": "delZ"
}

input_path = f"../../MITgcm/experiments/rotated_BBTRE_canyon-tracer-2xkappa/input/"
print(f"\n{input_path}")
new_ds = ds.copy()

for (k, v) in input_vars.items():
    # save input data as binary files
    newFile = open(f"{input_path}{v}.init", "wb")
    newFile.write(bytes(new_ds[k].values.astype('>f8')))
    newFile.close()

checklist_name = f"{input_path}checklist.txt"
if not(os.path.exists(checklist_name)):
    f = open(checklist_name, "w")
    f.write(checklist())
    f.close()
else:
    print(" (Checklist file already exists!)")

os.system(f"cp ./restart_with_higher_bottom-kappa.ipynb {input_path}");


../../MITgcm/experiments/rotated_BBTRE_canyon-tracer-2xkappa/input/
 (Checklist file already exists!)
